In [49]:
import os
import yaml
import pandas as pd

from pathlib import Path
# from sklearn.metrics import ConfusionMatrixDisplay

from ba_dev.utils import load_path_yaml
from ba_dev.eval_helpers import *

set_custom_plot_style()


In [50]:
paths = load_path_yaml('/cfs/earth/scratch/kraftjul/BA/data/path_config.yml')

path_to_models = paths['complete_models']


In [3]:
df = evaluate_all_runs(
    path_to_runs=path_to_models,
    metrics='balanced_accuracy_score',
    )

In [4]:
df

,model_name,pretrained,experiment_name,trainable_params,metric,fold,img_score,seq_score
0,resnet50,False,resNet50_v1_no_pretrained_cross_val,23516228,balanced_accuracy_score,0,0.984100,0.987187
1,resnet50,False,resNet50_v1_no_pretrained_cross_val,23516228,balanced_accuracy_score,1,0.988517,0.987694
2,resnet50,False,resNet50_v1_no_pretrained_cross_val,23516228,balanced_accuracy_score,2,0.984652,0.990435
3,resnet50,False,resNet50_v1_no_pretrained_cross_val,23516228,balanced_accuracy_score,3,0.979698,0.984942
4,resnet50,False,resNet50_v1_no_pretrained_cross_val,23516228,balanced_accuracy_score,4,0.987796,0.993614
5,efficientnet_b0,True,efficientnet_b0_v4_pretrained_cross_val,4012672,balanced_accuracy_score,0,0.992749,0.996595
6,efficientnet_b0,True,efficientnet_b0_v4_pretrained_cross_val,4012672,balanced_accuracy_score,1,0.994099,0.994376
7,efficientnet_b0,True,efficientnet_b0_v4_pretrained_cross_val,4012672,balanced_accuracy_score,2,0.992898,0.994796
8,efficientnet_b0,True,efficientnet_b0_v4_pretrained_cross_val,4012672,balanced_accuracy_score,3,0.985684,0.991318
9,efficientnet_b0,True,efficientnet_b0_v4_pretrained_cross_val,4012672,balanced_accuracy_score,4,0.994866,0.996577


In [56]:
table = (
    df
    .groupby(['model_name', 'pretrained'])
    .agg(
        trainable_params=('trainable_params', 'first'),
        img_score_mean=('img_score', 'mean'),
        img_score_std =('img_score', 'std'),
        seq_score_mean=('seq_score', 'mean'),
        seq_score_std =('seq_score', 'std'),
    )
    .reset_index()
    .sort_values(by='img_score_mean', ascending=False)
    ).rename(columns={
    'model_name': 'Model',
    'pretrained': 'Pretrained',
    'trainable_params': 'Params (M)',
    'img_score_mean': r'$\mu_{\mathrm{img}}$',
    'img_score_std': r'$\sigma_{\mathrm{img}}$',
    'seq_score_mean': r'$\mu_{\mathrm{seq}}$',
    'seq_score_std': r'$\sigma_{\mathrm{seq}}$',
})

table['Pretrained'] = table['Pretrained'].map({True: 'Yes', False: 'No'})
table['Params (M)'] = (table['Params (M)'] / 1_000_000).astype(int)

float_fmt = lambda x: f"{x:.4f}"

latex = table.to_latex(
    index=False,
    column_format='l c r c c c c',
    float_format=float_fmt,
    escape=False,
    caption='Balanced accuracy by model and pretraining arrangad by image score.',
    label='tab:bal_acc_by_model'
    )

with open(paths['table'] / 'bal_acc_by_model.tex', 'w') as f:
    f.write(latex)

In [55]:
print(latex)

\begin{table}
\caption{Aggregated scores by model and pretraining}
\label{tab:aggregated-scores}
\begin{tabular}{l c r c c c c}
\toprule
Model & Pretrained & Params (M) & $\mu_{\mathrm{img}}$ & $\sigma_{\mathrm{img}}$ & $\mu_{\mathrm{seq}}$ & $\sigma_{\mathrm{seq}}$ \\
\midrule
efficientnet_b0 & Yes & 4 & 0.9921 & 0.0037 & 0.9947 & 0.0022 \\
densenet169 & Yes & 12 & 0.9904 & 0.0044 & 0.9939 & 0.0016 \\
resnet50 & Yes & 23 & 0.9899 & 0.0036 & 0.9934 & 0.0019 \\
vit_b_16 & Yes & 85 & 0.9885 & 0.0052 & 0.9933 & 0.0023 \\
densenet169 & No & 12 & 0.9863 & 0.0059 & 0.9899 & 0.0017 \\
efficientnet_b0 & No & 4 & 0.9856 & 0.0049 & 0.9898 & 0.0026 \\
resnet50 & No & 23 & 0.9850 & 0.0035 & 0.9888 & 0.0033 \\
\bottomrule
\end{tabular}
\end{table}



In [26]:
agg.keys()

AttributeError: 'NoneType' object has no attribute 'keys'

In [ ]:
select_cols = ['model_name', 'pretrained', 'trainable_params', 'img_score_mean',
       'img_score_std', 'seq_score_mean', 'seq_score_std']

In [16]:
df2 = df.rename(columns={
    'model_name': 'Model',
    'pretrained': 'Pretrained',
    'trainable_params': '# Params',
    'img_score_mean': r'$\mu_{\mathrm{img}}$',
    'img_score_std': r'$\sigma_{\mathrm{img}}$',
    'seq_score_mean': r'$\mu_{\mathrm{seq}}$',
    'seq_score_std': r'$\sigma_{\mathrm{seq}}$',
})

# map True/False → Yes/No
df2['Pretrained'] = df2['Pretrained'].map({True: 'Yes', False: 'No'})

# format numbers
df2['# Params'] = df2['# Params'].map(lambda x: f"{x:,}")
float_fmt = lambda x: f"{x:.4f}"

# 2. Export to LaTeX
latex = df2.to_latex(
    index=False,
    header=df2.columns,
    column_format='l c r c c c c',
    float_format=float_fmt,
    escape=False,         # so that LaTeX math (e.g. \$\mu\$) passes through
    caption='Aggregated scores by model and pretraining',
    label='tab:aggregated-scores'
    )   

print(latex)

\begin{table}
\caption{Aggregated scores by model and pretraining}
\label{tab:aggregated-scores}
\begin{tabular}{l c r c c c c}
\toprule
Model & Pretrained & experiment_name & # Params & metric & fold & img_score & seq_score \\
\midrule
resnet50 & No & resNet50_v1_no_pretrained_cross_val & 23,516,228 & balanced_accuracy_score & 0 & 0.9841 & 0.9872 \\
resnet50 & No & resNet50_v1_no_pretrained_cross_val & 23,516,228 & balanced_accuracy_score & 1 & 0.9885 & 0.9877 \\
resnet50 & No & resNet50_v1_no_pretrained_cross_val & 23,516,228 & balanced_accuracy_score & 2 & 0.9847 & 0.9904 \\
resnet50 & No & resNet50_v1_no_pretrained_cross_val & 23,516,228 & balanced_accuracy_score & 3 & 0.9797 & 0.9849 \\
resnet50 & No & resNet50_v1_no_pretrained_cross_val & 23,516,228 & balanced_accuracy_score & 4 & 0.9878 & 0.9936 \\
efficientnet_b0 & Yes & efficientnet_b0_v4_pretrained_cross_val & 4,012,672 & balanced_accuracy_score & 0 & 0.9927 & 0.9966 \\
efficientnet_b0 & Yes & efficientnet_b0_v4_pretrained_cr